In [ ]:
!pip install -U huggingface_hub transformers langgraph langchain-core


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2


In [ ]:
!pip install -U langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install -U torch torchvision torchaudio


In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import json
from huggingface_hub import InferenceClient

from google.colab import userdata
from huggingface_hub import login
token = # put hugging face token here


if token:
    login(token=token, add_to_git_credential=False)
else:
    print("HF_TOKEN secret not found. Public models will still be searchable, but gated/private ones may fail.")


In [ ]:
# ----------------------
# Load catalogs
# ----------------------
with open("/content/catalog_utils.json", "r") as f:
    tools_catalog = json.load(f)
with open("/content/catalog_agents2.json", "r") as f:
    agents_catalog = json.load(f)

## Utility Functions

In [ ]:
from typing import TypedDict, List, Dict, Any, Optional # Import Optional
import re
import json
import logging
import os
# ----------------------
# LLM client
# ----------------------
client = InferenceClient(provider="novita", api_key=token)

# ----------------------
# UNIFIED LLM CALLER
# ----------------------
def call_llm(prompt, user_content, few_shot_file=None, model="openai/gpt-oss-120b", temperature=0):
    """Wrapper to call LLM with optional few-shot examples stored in JSON."""
    few_shots = []
    if few_shot_file:
        with open(few_shot_file, "r") as f:
            few_shots = json.load(f)

    messages = [{"role": "system", "content": prompt}] + few_shots + [{"role": "user", "content": user_content}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=False
    )
    return response.choices[0].message["content"]

# ----------------------
# LLM helper for JSON rewrite (clarification)
# ----------------------
def call_llm_json_rewrite(plan_json: Dict[str, Any], human_feedback: str,
                          few_shot_file: str = None,
                          model: str = "openai/gpt-oss-120b",
                          temperature: float = 0.0) -> Dict[str, Any]:
    """
    Ask the LLM to rewrite the given plan JSON based on human feedback.
    This function forces the model (via a system prompt) to return JSON only.
    Returns parsed JSON (dict) or raises ValueError if parsing fails.
    """
    messages = [{"role": "system", "content": CLARIFICATION_REWRITE_PROMPT}]
    if few_shot_file:
        try:
            with open(few_shot_file, "r", encoding="utf-8") as f:
                few_shots = json.load(f)
            messages += few_shots
        except Exception:
            pass

    messages.append({"role": "user", "content": f"Original plan JSON:\n{json.dumps(plan_json, indent=2)}"})
    messages.append({"role": "user", "content": f"Human feedback (natural language):\n{human_feedback}\n\nRewrite the plan JSON accordingly. Output JSON only."})

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=False
    )
    text = resp.choices[0].message["content"]

    # Parse JSON: best-effort
    try:
        parsed = json.loads(text)
        return parsed
    except Exception as e:
        # If parsing failed, raise so caller can handle (e.g., prompt human to rephrase)
        raise ValueError(f"LLM did not return valid JSON. Error: {e}\nRaw output:\n{text}")

# Custom LLM wrapper class
from langchain.llms.base import LLM
class CustomLLM(LLM):
    model: str = "openai/gpt-oss-120b"
    temperature: float = 0
    few_shot_file: Optional[str] = None

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # For agent use, we typically don't need a separate system prompt
        # as the agent framework handles that
        return call_llm("", prompt, self.few_shot_file, self.model, self.temperature)

# Initialize the custom LLM
llm = CustomLLM(
    model="openai/gpt-oss-120b",
    temperature=0,
    few_shot_file=None  # Set this if you have few-shot examples
)



In [ ]:
def parse_prompt(prompt: str) -> Dict[str, Optional[str]]:
    result = {
        "audio_dir": None,
        "ground_truth_csv": None,
        "lang_code": None
    }
    path_pattern = r"['\"]?(/[^'\"]+/[^'\"]+/)['\"]?"
    csv_pattern = r"['\"]?(/[^'\"]+\.csv)['\"]?"
    lang_pattern = r"\b(bn|gu|hi|kn|ml|mr|pa|sd|si|ta|te|ur)\b"
    audio_dir_match = re.search(path_pattern, prompt)
    if audio_dir_match:
        result["audio_dir"] = audio_dir_match.group(1)
    csv_match = re.search(csv_pattern, prompt)
    if csv_match:
        result["ground_truth_csv"] = csv_match.group(1)
    lang_match = re.search(lang_pattern, prompt, re.IGNORECASE)
    if lang_match:
        result["lang_code"] = lang_match.group(1).lower()
    if not result["audio_dir"] and not result["ground_truth_csv"]:
        llm_prompt = f"""Extract the following from the prompt:
        1. Audio directory path (e.g., /path/to/audio/)
        2. Ground truth CSV path (e.g., /path/to/file.csv)
        3. Language code (e.g., hi, te)
        If any are unclear, return None for that field.
        Return a JSON object with keys 'audio_dir', 'ground_truth_csv', 'lang_code'.
        Prompt:
        """
        try:
            response = call_llm(llm_prompt, prompt)    #llm.invoke(llm_prompt)
            content = response.strip() # Use response from call_llm
            if content.startswith('```json'):                                                                       #markdown format correction
                content = content.replace('```json', '').replace('```', '').strip()
            llm_result = json.loads(content)
            for key in result:
                if not result[key]:
                    result[key] = llm_result.get(key)
        except Exception as e:
            logging.error(f"LLM prompt parsing failed: {e}")                                                         # LLM failsafe
            if audio_dir_match:
                result["audio_dir"] = audio_dir_match.group(1)
            elif "Audio_dir" in prompt:
                audio_dir_match = re.search(r"Audio_dir\s*=\s*['\"]?(/[^'\"]+/)['\"]?", prompt)
                if audio_dir_match:
                    result["audio_dir"] = audio_dir_match.group(1)

    if result["audio_dir"] and not os.path.isdir(result["audio_dir"]):
        logging.error(f"Invalid audio directory: {result['audio_dir']}")
        result["audio_dir"] = None
    if result["ground_truth_csv"] and not os.path.isfile(result["ground_truth_csv"]):
        logging.error(f"Invalid ground truth CSV: {result['ground_truth_csv']}")
        result["ground_truth_csv"] = None
    return result

In [ ]:
import os
import logging
import torch
import librosa
import pandas as pd
from pydub import AudioSegment
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, AutoTokenizer

def transcribe_audio(audio_path, source_lang):
    model_ids = {
        "Hindi": "ai4bharat/indicwav2vec-hindi",
        "Tamil": "Harveenchadha/vakyansh-wav2vec2-tamil-tam-250",
        "Sanskrit": "addy88/wav2vec2-sanskrit-stt",
        "Marathi": "ai4bharat/indicwav2vec-marathi",
        "Telugu": "ai4bharat/indicwav2vec-telugu",
    }
    if source_lang not in model_ids:
        raise ValueError(f"Unsupported language for ai4bharat/indicwav2vec2: {source_lang}")
    model_id = model_ids[source_lang]
    processor = Wav2Vec2Processor.from_pretrained(model_id)
    model = Wav2Vec2ForCTC.from_pretrained(model_id)
    print(f"Transcribing {audio_path} using {model_id}")
    audio_array, sr = librosa.load(audio_path, sr=16000)
    inputs = processor(audio_array, sampling_rate=sr, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        emission = torch.nn.functional.log_softmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    transcript_words = transcription[0].strip().split()
    print("TRANSCRIPT:", transcript_words)
    return " ".join(transcript_words)


def transcribe_folder_to_csv(folder_path: str, source_language: str):
    output_path = os.path.join(folder_path, "indicconf_hypothesis.csv")
    if os.path.exists(output_path):
        logging.info(f"Skipping transcription: File already exists at {output_path}")
        return output_path

    results = []
    for filename in os.listdir(folder_path):
        audio_path = os.path.join(folder_path, filename)
        file_size = os.path.getsize(audio_path)
        if file_size == 0:
            logging.warning(f"Skipping {filename}: File size is 0 bytes")
            continue

        delete_after = False

        if filename.lower().endswith(".mp3"):
            try:
                wav_filename = os.path.splitext(filename)[0] + ".wav"
                wav_path = os.path.join(folder_path, wav_filename)
                audio = AudioSegment.from_mp3(audio_path)
                audio.export(wav_path, format="wav")
                logging.info(f"Converted {filename} to {wav_filename}")
                audio_path = wav_path
                transcribe_filename = filename
                delete_after = True
            except Exception as e:
                logging.error(f"Failed to convert {filename} to .wav: {e}")
                continue

        elif not filename.lower().endswith(".wav"):
            logging.warning(f"Skipping {filename}: Not a .wav or .mp3 file")
            continue
        else:
            transcribe_filename = filename

        try:
            hypothesis = transcribe_audio(audio_path, source_language)
            results.append({
                "Filename": transcribe_filename,
                "Indiconformer_Hypothesis": hypothesis
            })
            logging.info(f"Transcribed {transcribe_filename}: {hypothesis[:50]}...")
        except Exception as e:
            logging.error(f"Failed to transcribe {transcribe_filename}: {e}")
        finally:
            if delete_after:
                try:
                    os.remove(audio_path)
                    logging.info(f"Deleted temporary file: {audio_path}")
                except Exception as e:
                    logging.error(f"Failed to delete temporary .wav file {audio_path}: {e}")

    if not results:
        logging.error("No valid transcriptions generated")
        return "Error: No valid transcriptions"

    df = pd.DataFrame(results)
    df.to_csv(output_path, index=False)
    logging.info(f"Transcriptions saved to {output_path}")
    return output_path

## Prompts

In [ ]:
TASK_DECOMPOSITION_PROMPT = """
You are a task decomposition system that maps a user's request about an audio dataset to:
1. The list of EXISTING_TASKS (from a fixed predefined agent library, outputting function names)
2. Any NEW_TASKS not covered by the existing library

Your job:
- Read the user’s request
- Select **all** predefined agents whose meaning applies, regardless of exact wording
- Add any new tasks that are not already covered by the predefined list

## OUTPUT FORMAT
Respond ONLY with a single JSON object that matches this schema exactly:
{
  "EXISTING_TASKS": ["agent_function_name", "agent_function_name", ...],
  "NEW_TASKS": [
    {
      "name": "string",
      "description": "string"
    },
    ...
  ]
}
No extra text, comments, or explanations — only valid JSON.

## SEMANTIC DEFINITIONS FOR EXISTING AGENTS
Always select an agent if the request matches the meaning below, even if different words are used.

"transcription_func": Transcription — Creating a written transcript of spoken content in the audio.
"num_speaker_func": Speaker identification/diarization — Determining who is speaking, separating speech by speaker, counting speakers, or labeling speaker turns.
"transcript_quality_agent": Transcript quality assessment — Evaluating transcription accuracy, completeness, and formatting.
"character_agent": Character/graphene calculation — Counting characters or graphemes in the transcript.
"vocab_agent": Vocabulary calculation — Counting and listing unique words in the transcript.
"language_verification_agent": Language verification — Checking if the transcript matches the expected language.
"audio_length_agent": Audio length calculation — Determining the total duration of audio files.
"silence_vad_func": Silence detection — Identifying periods of no speech in the audio using VAD.
"english_word_count_agent": English word counter — Counting English words in the transcript.
"ctc_score_agent": CTC score calculation — Computing CTC-based alignment score between audio and transcript.
"upsampling_agent": Upsampling check — Detecting artificially upsampled audio.
"valid_speaker_agent": Speaker validity — Checking if speakers are new or previously known.
"domain_checker_agent": Domain detection — Determining the speech dataset’s domain (e.g., medical, legal).
"audio_transcript_matching_agent": Forced alignment — Mapping transcript text to corresponding audio segments.
"language_identification_indiclid_agent": Language identification — Detecting the language(s) spoken in the audio.
"normalization_remove_tags_agent": Transcript normalization — Removing HTML/XML tags and formatting from transcriptions.
"llm_score_agent": Coherence/fluency scoring — Using LLM-as-a-Judge to score transcript fluency (1–10 scale).
"transliteration_agent": Transliteration — Converting Roman script words to native script.
"corruption_agent": Audio corruption detection — Detecting distortion, clipping, or other signal issues.
"extension_agent": Audio format verification — Checking that files have correct extensions (e.g., .wav).
"sample_rate_agent": Sample rate verification — Ensuring audio sample rates match requirements.
"speaker_duration_agent": Per-speaker duration measurement — Calculating total speaking time for each speaker.
"utterance_duplicate_checker_agent": Utterance duplicate check — Detecting duplicate transcriptions.
"wer_computation_agent": WER computation — Calculating Word Error Rate between reference and hypothesis transcripts.

## RULES
- Always match based on meaning, not exact wording.
- If a request includes multiple actions, include **all applicable agents**.
- Do not reword existing agent names; only output their function names in EXISTING_TASKS.
- Any requirement not covered above must be added to NEW_TASKS.
- NEW_TASKS must have a concise `name` and a clear `description`.
"""


TOOL_PLANNER_SYSTEM_PROMPT = """
You are a Tool Planning Agent.

# Inputs Provided to You:
1. A catalog of existing agents and tool functions with their:
   - descriptions, inputs, outputs, and dependencies.
2. A list of tasks that must be mapped into tool/agent usage.
3. The PipelineState schema (a state dictionary in LangGraph) that defines available state variables.

# Important Convention:
- Every tool or agent function must follow this signature:
    def tool_name(state: PipelineState) -> PipelineState

- Tools **do not take individual parameters**. They always receive the entire state dictionary and must:
  1. Read the required fields from the state (e.g., `audio_dir = state.get("audio_dir")`).
  2. Perform the operation.
  3. Write results back into the state using the correct field (e.g., `state["num_speakers"] = value`).
  4. Return the updated state.

# PipelineState fields (shared memory between tools/agents):
- audio_dir: str → Path to folder containing audio files
- ground_truth_csv: str → Path to ground-truth CSV file if provided
- transc_csv: str → Path to transcript CSV file created by transcript_func
- test_transc_csv: str → Path to test transcript CSV file for validating new tools/agents
- test_audio_dir: str → Path to test audio directory for validating new tools/agents
- lang_code: str → Language code for processing
- user_request: str → Raw natural language request from the user
- task_decomposition: dict → Structured breakdown of tasks
- task_list_for_planner: list[str] → Flattened list of tasks passed to planner
- tool_plan: dict → JSON representation of tool/agent plan
- clarification_round: int → Current clarification round
- clarification_history: list[dict] → Log of clarification interactions
- clarification_done: bool → Whether clarification is finished
- human_feedback: str → Freeform feedback text from user

# For each task you must decide:
- Can it be fulfilled with an existing tool/agent?
- Does an existing tool/agent need modification (small changes in I/O, additional checks)?
- Or must a new tool/agent be created?
- If a new tool/agent is proposed, you MUST decide if it is Achievable or Not Achievable:
  - **Achievable = "Yes"** → provide technical justification (libraries).
  - **Achievable = "No"** → provide reason + specify the missing capability (e.g., requires publicly available models, requires raw audio sentiment analysis, multimodal pretraining, unavailable model).
  - If new tasks involves loading publicly available models , they SHOULD NOT be classified as ACHIEVABLE.


# JSON Output Format (MANDATORY):
{
  "PLAN": [
    {
      "task": "string - copy of the task description",
      "use_existing": ["ToolName1", "ToolName2"],
      "modify_existing": [
        {
          "tool": "ToolName",
          "modification_description": "Concrete explanation of what needs to be changed"
        }
      ],
      "create_new": [
        {
          "name": "string - proposed tool/agent name",
          "description": "Detailed description of functionality and role",
          "Achievable" : "Justification as to why it is `achievable` or `not achievable`"
          "input_spec": "PipelineState - uses fields: [field1, field2]" ,
          "output_spec": "PipelineState - updates fields: [field3, field4]" ,
          "tool_dependencies": ["list","of","functions","or","libraries","needed"],
          "agent_dependencies": ["list","of","agents","needed","if","any"]
        }
      ]
    }
  ]
}

# Rules (STRICT):
1. Always describe I/O in terms of **PipelineState fields** if the new task is achievable:
   - input_spec must list which fields are *read* from state.
   - output_spec must list which fields are *written* to state.

   Example:
   "input_spec": "PipelineState - uses fields: [audio_dir]",
   "output_spec": "PipelineState - updates fields: [num_speakers]"

2. Be precise in `modification_description`:
   - Example: "Modify transcript_func to also read lang_code from PipelineState and handle multilingual input."

3. Each task must have at least one of: `use_existing`, `modify_existing`, or `create_new`.

4. `tool_dependencies`: only functions/modules/LLM utilities required for implementation.
   `agent_dependencies`: only existing higher-level agents that this tool relies on.

5. If a task involves evaluation, scoring, or subjective judgment prefer LLM: (example sentiment on text)
   - First check if a scoring tool exists.
   - If not, create a new scoring agent that wraps an LLM function.
   - Always state: "Scoring is performed by an LLM with a well-defined prompt."

6. Multiple tasks → multiple objects in the "PLAN" list.
   Do not merge unrelated tasks.

# Example:
{
  "PLAN": [
    {
      "task": "Count number of speakers in audio",
      "use_existing": [],
      "modify_existing": [],
      "create_new": [
        {
          "name": "num_speaker_func",
          "description": "Counts the number of distinct speakers in the audio files located in audio_dir.",
          "input_spec": "PipelineState - uses fields: [audio_dir]",
          "output_spec": "PipelineState - updates fields: [num_speakers]",
          "tool_dependencies": ["pyannote.audio"],
          "agent_dependencies": []
        }
      ]
    }
  ]
}
"""


# TOOL_PLANNER_SYSTEM_PROMPT = """
# You are a Tool Planning Agent.

# # You are given:
# 1. A catalog of existing agents and tools functions with their descriptions, inputs, outputs, and dependencies.
# 2. A list of tasks to perform.
# 3. The PipelineState schema that defines available state variables in the LangGraph pipeline.

# # PipelineState fields:
# - audio_dir: str → Path to folder containing audio files
# - ground_truth_csv: str → Path to ground-truth CSV file if provided
# - transc_csv: str → Path to transcript CSV file created by transcript_func
# - test_transc_csv: str → Path to test transcript CSV file for validating new tools/agents
# - test_audio_dir: str → Path to test audio directory for validating new tools/agents
# - lang_code: str → Language code for processing
# - user_request: str → Raw natural language request from the user
# - task_decomposition: dict → Structured breakdown of tasks
# - task_list_for_planner: list[str] → Flattened list of tasks passed to planner
# - tool_plan: dict → JSON representation of tool/agent plan
# - clarification_round: int → Current clarification round
# - clarification_history: list[dict] → Log of clarification interactions
# - clarification_done: bool → Whether clarification is finished
# - human_feedback: str → Freeform feedback text from user

# # For each task, decide:
# - If an existing tool/agent can be used as-is
# - If an existing tool/agent can be modified to achieve it
# - If a new tool/agent must be created

# # Output JSON only in this format:
# {
#   "PLAN": [
#     {
#       "task": "string",
#       "use_existing": ["ToolName1", "ToolName2"],
#       "modify_existing": [
#         {
#           "tool": "ToolName",
#           "modification_description": "string"
#         }
#       ],
#       "create_new": [
#         {
#           "name": "string",
#           "description": "string",
#           "input_spec": "string - must reference a PipelineState field when possible",
#           "output_spec": "string - must reference a PipelineState field when possible",
#           "tool_dependencies": ["list","of","strings","of","tool","names","from","catalog"],
#           "agent_dependencies": ["list","of","strings","of","agent","names","from","catalog"]
#         }
#       ]
#     }
#   ]
# }

# # Rules:
# - Always prefer PipelineState fields for input_spec and output_spec (e.g., `"string - Path to folder containing audio files (audio_dir) from PipelineState"`).
# - If no relevant PipelineState field exists, allow a general spec (e.g., "string - User query").
# - Only reference tools and agents from the provided catalog unless creating new.
# - Match dependencies based on functionality, not just wording similarity.
# - input_spec and output_spec :
#     - `"type - meaning + PipelineState + field"` when applicable. important to mention state name
#     - `"type - meaning "`
# - tool_dependencies: list only function names, modules, llm or libraries directly needed for the implementation.
# - agent_dependencies: list existing agents that might be needed before/after implementing this tool/agent.
# - If a task involves evaluation, scoring, grading, or subjective judgment (e.g., relevance, sentiment, fluency, coherence), check for a scoring tool in the catalog first.
#   - If none exists, create a new LLM-based scoring agent that uses an existing LLM utility function as the primary dependency.
#   - Always specify in the description that scoring is performed by an LLM with a well-defined prompt.

# """


In [ ]:
CLARIFICATION_REWRITE_PROMPT = """
You are a JSON rewrite assistant.

You will be given:
1) Original tool plan JSON.
2) Natural language human feedback.

Task:
- Incorporate the human feedback into the original plan by modifying only the necessary parts.
- Keep the exact JSON structure:
{
  "PLAN": [
    {
      "task": "string",
      "use_existing": ["ToolName1", "ToolName2"],
      "modify_existing": [
        {"tool": "ToolName", "modification_description": "string"}
      ],
      "create_new": [
        {
          "name": "string",
          "description": "string",
          "input_spec": "string",
          "output_spec": "string",
          "tool_dependencies": ["list","of","strings"],
          "agent_dependencies": ["list","of","strings"]
        }
      ]
    }
  ]
}
Field Descriptions:
- **task**: A short description of the specific task to be performed.
- **use_existing**: List of existing tools that will be directly used without modification.
- **modify_existing**: List of modifications to existing tools (tool name + description of change).
- **create_new**: List of entirely new tools that need to be created. Each new tool requires:
  - **name**: Unique name of the new tool.
  - **description**: Purpose and functionality of the tool.
  - **input_spec**: Description of input format and requirements.
  - **output_spec**: Description of output format and expectations.
  - **tool_dependencies**: List of other tools this tool depends on.
  - **agent_dependencies**: List of agent processes or agents this tool depends on.

**Important:**
- Output **only valid JSON** (no extra text, no commentary).
- Preserve unchanged entries unless the human feedback requires a change.
- If feedback asks to merge or remove items, reflect that exactly in the JSON.
- The three fields **use_existing, modify_existing, create_new** are exhaustive.

"""

## Python Tool

In [ ]:
from langchain.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.agents import initialize_agent, AgentType

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)
tools = [repl_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

/tmp/ipython-input-406648470.py:12: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


## LangGraph Agent

In [ ]:
from typing import TypedDict, List, Dict, Any
# ----------------------
# State type
# ----------------------
class PipelineState(TypedDict, total=False):
    audio_dir: str
    ground_truth_csv: str
    transc_csv: str
    lang_code: str
    user_request: str
    task_decomposition: Dict[str, Any]
    task_list_for_planner: List[str]
    tool_plan: Dict[str, Any]
    clarification_round: int
    clarification_history: List[Dict[str, Any]]
    clarification_done: bool
    human_feedback: str  # Add this field

In [ ]:
# ----------------------
# LANGGRAPH NODES
# ----------------------

def task_decomposition_node(state):
    """Node 1: Decompose user request into tasks."""
    print("---Task Decomposition---")
    user_request = state["user_request"]
    print(f"--------------------Calling LLM--------------------")
    task_decomp_output = call_llm(
        TASK_DECOMPOSITION_PROMPT,
        user_request,
        few_shot_file="/content/task_decomposer.json"
    )
    print(f"--------------------LLM output--------------------")
    print(task_decomp_output)

    tasks_data = json.loads(task_decomp_output)
    task_list_for_planner = list(tasks_data["EXISTING_TASKS"]) + [t["name"] for t in tasks_data["NEW_TASKS"]]

    return {
        "task_decomposition": tasks_data,
        "task_list_for_planner": task_list_for_planner
    }


def tool_planning_node(state):
    """Node 2: Plan tools/agents based on tasks."""
    print("---Tool Planning---")
    tool_planner_input = json.dumps({
        "tools_catalog": tools_catalog,
        "agents_catalog": agents_catalog,
        "tasks": state["task_list_for_planner"]
    }, indent=2)

    print(f"--------------------Calling LLM--------------------")
    tool_plan_output = call_llm(
        TOOL_PLANNER_SYSTEM_PROMPT,
        tool_planner_input,
        few_shot_file="/content/tool_planner.json"
    )
    raw = tool_plan_output.strip()
    print(f"Tool plan: {raw}")

    try:
        tool_plan = json.loads(raw)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", raw, re.DOTALL)
        if match:
            try:
                tool_plan = json.loads(match.group(0))
            except json.JSONDecodeError:
                raise ValueError(f"Extracted JSON block is invalid: {match.group(0)}")

    return {"tool_plan": tool_plan}

# ----------------------
# Node 3: Clarification (HITL) - CORRECTED VERSION
# ----------------------

def clarification_request_node(state):
    """
    Ask human to review tool plan and provide feedback.
    Uses interrupt() to pause execution and wait for human input.
    """
    print("---Clarification Request---")
    current_plan = state.get("tool_plan", {})
    round_number = state.get("clarification_round", 0) + 1

    prompt_text = (
        f"=== Clarification Round {round_number} ===\n\n"
        "Please review the current tool plan below and respond with:\n"
        "- 'approve' or 'ok' to accept the plan, OR\n"
        "- Natural language instructions to change the plan.\n\n"
        "Current plan:\n"
        f"{json.dumps(current_plan, indent=2)}\n\n"
        "Your feedback:"
    )

    # Use interrupt() to pause and wait for human feedback
    feedback = interrupt(prompt_text)

    # Return the feedback to be stored in state
    return {
        "human_feedback": feedback,
        "clarification_round": round_number
    }


def clarification_process_node(state):
    """
    Process human feedback, update plan if needed.
    """
    print("---Processing Clarification---")
    current_plan = copy.deepcopy(state.get("tool_plan", {}))
    feedback_text = (state.get("human_feedback") or "").strip()
    round_number = state.get("clarification_round", 0)

    # If approved → finish
    if feedback_text.lower() in {"ok", "approve", "approved", "yes", "looks good"}:
        history_entry = {
            "round": round_number,
            "feedback": feedback_text,
            "action": "approved",
            "plan_before": current_plan,
            "plan_after": current_plan
        }
        clarification_history = state.get("clarification_history", []) + [history_entry]
        return {
            "clarification_history": clarification_history,
            "clarification_done": True
        }

    # Otherwise → rewrite plan with LLM
    rewritten_plan = call_llm_json_rewrite(
        plan_json=current_plan,
        human_feedback=feedback_text,
        few_shot_file=None,
        model="openai/gpt-oss-20b",
        temperature=0.0
    )

    history_entry = {
        "round": round_number,
        "feedback": feedback_text,
        "action": "rewritten",
        "plan_before": current_plan,
        "plan_after": rewritten_plan
    }

    return {
        "tool_plan": rewritten_plan,
        "clarification_history": state.get("clarification_history", []) + [history_entry],
        "clarification_done": False
    }


In [ ]:
def transcription_func(state: PipelineState):
    audio_dir = state.get('audio_dir')
    if not audio_dir or not os.path.isdir(audio_dir):
        logging.error(f"Invalid audio directory for transcription: {audio_dir}")
        return {"A": "Error: Invalid audio directory"}
    logging.info("Running Transcription")
    result = transcribe_folder_to_csv(audio_dir, source_language="Hindi")
    return {"transc_csv": result} #, "audio_dir": audio_dir

In [ ]:
import re
import json
import copy
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END
from langgraph.constants import START
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver

def clarification_condition(state):
    """Decide whether to continue or end clarification loop."""
    if state.get("clarification_done", False) or state.get("clarification_round", 0) >= 3:
        return END
    return "clarification_request"


# Build state graph
graph = StateGraph(PipelineState)

# Add nodes
graph.add_node("task_decomposition", task_decomposition_node)
graph.add_node("tool_planning", tool_planning_node)
graph.add_node("clarification_request", clarification_request_node)
graph.add_node("clarification_process", clarification_process_node)

# Add edges
graph.add_edge(START, "task_decomposition")
graph.add_edge("task_decomposition", "tool_planning")
graph.add_edge("tool_planning", "clarification_request")
graph.add_edge("clarification_request", "clarification_process")

# Add conditional edge for clarification loop
graph.add_conditional_edges("clarification_process", clarification_condition)

# CRITICAL: Add checkpointer for human-in-the-loop to work
memory = InMemorySaver()
app = graph.compile(checkpointer=memory)

In [ ]:
def run_pipeline_with_hitl(user_request: str):
    """Run the pipeline with proper human-in-the-loop handling."""

    # Create thread configuration
    thread_config = {"configurable": {"thread_id": "pipeline_thread_1"}}

    # Initial state
    initial_state = {"user_request": user_request, "clarification_round": 0}

    print("Starting pipeline execution...")

    # Stream the graph execution
    for event in app.stream(initial_state, thread_config, stream_mode="updates"):
        print(f"Event: {event}")

        # Check if we hit an interrupt
        if "__interrupt__" in event:
            interrupt_data = event["__interrupt__"][0]
            print(f"\n HUMAN INPUT REQUIRED:")
            print(f"Prompt: {interrupt_data.value}")

            # Get human feedback
            human_feedback = input("\nYour feedback: ").strip()

            # Resume execution with the feedback
            print(f"\n Resuming with feedback: {human_feedback}")

            # Continue streaming with the human feedback
            for resume_event in app.stream(
                Command(resume=human_feedback),
                thread_config,
                stream_mode="updates"
            ):
                print(f"Resume Event: {resume_event}")

                # Handle nested interrupts (if clarification loops)
                if "__interrupt__" in resume_event:
                    nested_interrupt = resume_event["__interrupt__"][0]
                    print(f"\n ADDITIONAL INPUT REQUIRED:")
                    print(f"Prompt: {nested_interrupt.value}")

                    nested_feedback = input("\nYour feedback: ").strip()
                    print(f"\n Resuming with feedback: {nested_feedback}")

                    # Continue with nested feedback
                    for final_event in app.stream(
                        Command(resume=nested_feedback),
                        thread_config,
                        stream_mode="updates"
                    ):
                        print(f"Final Event: {final_event}")

            break  # Exit the main loop after handling interrupt

    # Get final state
    final_state = app.get_state(thread_config)
    print("\n=== Final State ===")
    print(json.dumps(final_state.values, indent=2))

    return final_state

In [ ]:
user_request = ("I have audio file directory at /content/audios and i need to generate  transcript of each audio, and accent of the audio .") # sentiment of the transcript


In [ ]:
run_pipeline_with_hitl(user_request)

Starting pipeline execution...
---Task Decomposition---
--------------------Calling LLM--------------------
--------------------LLM output--------------------
{
  "EXISTING_TASKS": ["transcription_func"],
  "NEW_TASKS": [
    {
      "name": "accent_detection_agent",
      "description": "Detect and label the accent(s) present in each audio file or speaker."
    }
  ]
}
Event: {'task_decomposition': {'task_decomposition': {'EXISTING_TASKS': ['transcription_func'], 'NEW_TASKS': [{'name': 'accent_detection_agent', 'description': 'Detect and label the accent(s) present in each audio file or speaker.'}]}, 'task_list_for_planner': ['transcription_func', 'accent_detection_agent']}}
---Tool Planning---
--------------------Calling LLM--------------------
Tool plan: {
  "PLAN": [
    {
      "task": "transcription_func",
      "use_existing": ["transcription_func"],
      "modify_existing": [],
      "create_new": []
    },
    {
      "task": "accent_detection_agent",
      "use_existing": [],

StateSnapshot(values={'user_request': 'I have audio file directory at /content/audios and i need to generate  transcript of each audio, and accent of the audio .', 'task_decomposition': {'EXISTING_TASKS': ['transcription_func'], 'NEW_TASKS': [{'name': 'accent_detection_agent', 'description': 'Detect and label the accent(s) present in each audio file or speaker.'}]}, 'task_list_for_planner': ['transcription_func', 'accent_detection_agent'], 'tool_plan': {'PLAN': [{'task': 'transcription_func', 'use_existing': ['transcription_func'], 'modify_existing': [], 'create_new': []}, {'task': 'accent_detection_agent', 'use_existing': [], 'modify_existing': [], 'create_new': [{'name': 'accent_detection_agent', 'description': 'Detects speaker accent characteristics in each audio file of a speech dataset. The agent reads the audio directory, processes each file with an accent classification model, and writes a CSV summarizing filename, detected accent label, confidence score, and any model metadata.

In [ ]:
thread_config = {"configurable": {"thread_id": "pipeline_thread_1"}}

# Get current state
state = app.get_state(thread_config)
current_values = state.values

# Access specific elements
user_request = current_values.get("user_request")
tool_plan = current_values.get("tool_plan")
task_decomposition = current_values.get("task_decomposition")
clarification_round = current_values.get("clarification_round", 0)
clarification_done = current_values.get("clarification_done", False)

print(f"Current round: {clarification_round}")
print(f"Tool plan: {json.dumps(tool_plan, indent=2)}")

Current round: 2
Tool plan: {
  "PLAN": [
    {
      "task": "transcription_func",
      "use_existing": [
        "transcription_func"
      ],
      "modify_existing": [],
      "create_new": []
    },
    {
      "task": "sentiment_analysis",
      "use_existing": [],
      "modify_existing": [],
      "create_new": [
        {
          "name": "sentiment_analysis_agent",
          "description": "Analyzes sentiment (positive, neutral, negative) of each transcription in the CSV produced by the transcription step. The agent reads the path stored in PipelineState['transc_csv'], loads the CSV, sends each transcript to an LLM with a sentiment\u2011classification prompt, collects the LLM responses, adds a new column `sentiment` to the dataframe, writes the enriched CSV to a new file, and updates PipelineState['sentiment_output'] with the path of the generated CSV.",
          "Achievable": "Sentiment can be reliably obtained by prompting an LLM (e.g., GPT\u20114) with a clear instructi

In [ ]:
# workflow agent - > ask which tools to execute
#

In [ ]:
import os
import json
import logging
from typing import Dict, Any

def generate_agent_function_code(task_spec: Dict[str, Any]) -> str:
    """
    Generate a complete agent function using LLM for intelligent code generation.
    """

    function_name = task_spec.get("name")

    # Use LLM to generate the entire function intelligently
    generation_prompt = f"""
You need to generate a Python function similar to this example pattern:

```python
def corruption_agent(state: PipelineState) -> PipelineState:
    audio_dir = state.get('audio_dir')
    if not audio_dir or not os.path.isdir(audio_dir):
        logging.error(f"Invalid audio directory for corruption check: {{audio_dir}}")
        return {{"corruption_output": "Invalid: No audio directory"}}

    task_prompt = \"\"\"You are given a folder with audios at this path: {{audio_dir}}.
    Write a Python script to do the following and then execute it using [python_repl]:
    - Import the 'os' module.
    - Create a CSV named 'corruption_check.csv' in the same directory.
    - The CSV should have columns: Filename, Status.
    - For each audio file in the folder:
        - Use os.path.getsize(file_path) to get the file size.
        - If the file size is 0 KB, mark its 'Status' column as 'corrupt'.
        - If the file size is of .mp3 format, mark its 'Status' column as 'corrupt'.
        - Otherwise, mark its 'Status' column as 'valid'.
    - Save the CSV.
    Finally, respond with "Success" if all files are valid, otherwise respond with "Invalid".
    \"\"\"

    try:
        response = agent.invoke(task_prompt)
        return {{"corruption_output": response}}
    except Exception as e:
        logging.error(f"Corruption check failed: {{e}}")
        return {{"corruption_output": f"Error: {{e}}"}}
```

Now generate a similar function for this task specification:
- Function name: {function_name}
- Description: {task_spec.get("description")}
- Input specification: {json.dumps(task_spec.get("input_spec", {}), indent=2)}
- Output specification: {task_spec.get("output_spec", "")}
- Dependencies: {task_spec.get("python_dependencies", [])}
- Error handling: {task_spec.get("error_handling", "")}

Requirements:
1. PLease analyze the CSV and make no assumptions about column names
2. Analyze the task description to determine the appropriate approach:
  - If task requires AI analysis, judgment, classification, or content understanding → Use hybrid approach
  - If task is purely computational, file operations, or data processing → Use python_repl only
3. For hybrid approach tasks, structure the task_prompt with clear steps:
  - "Step 1: Use [python_repl] to [data operations]"
  - "Step 2: For each item, make LLM calls yourself (NOT through python_repl) to [analyze/classify/judge]"
  - "Step 3: Use [python_repl] to [save/write results]"
4. For python-only tasks, use the corruption_agent pattern with only [python_repl] instructions
5. Extract appropriate state keys based on the input specification description
6. Include proper validation for extracted state values (file/directory existence checks)
7. Include comprehensive error handling and logging
8. Return a dictionary with descriptive output key (function_name + "_output")
9. Make the task_prompt extremely specific with exact step-by-step instructions, not generic descriptions
10. Ensure the task_prompt is actionable and executable by the agent

Generate ONLY the complete function code, no explanations.
"""

    function_code = call_llm("", generation_prompt, model="openai/gpt-oss-120b", temperature=0.1)
    return function_code.strip()


In [ ]:
# ----------------------
# EXAMPLE USAGE
# ----------------------

sentiment_task ={
          "name": "sentiment_analysis_agent",
          "description": "Analyzes sentiment (positive, neutral, negative) of each transcription in the CSV produced by the transcription step. The agent reads the path stored in PipelineState['transc_csv'], loads the CSV, sends each transcript to an LLM with a sentiment\u2011classification prompt, collects the LLM responses, adds a new column `sentiment` to the dataframe, writes the enriched CSV to a new file, and updates PipelineState['sentiment_output'] with the path of the generated CSV.",
          "Achievable": "Sentiment can be reliably obtained by prompting an LLM (e.g., GPT\u20114) with a clear instruction and parsing the returned label. The required libraries (pandas for CSV handling and the existing `llm` tool for LLM calls) are already available in the environment.",
          "input_spec": "PipelineState - uses fields: [transc_csv]",
          "output_spec": "PipelineState - updates fields: [sentiment_output]",
          "tool_dependencies": [
            "llm",
            "pandas"
          ],
          "agent_dependencies": []
        }

# Test validation state
test_state = {
    "csv_path": "/path/to/test_transcript.csv",
    "test_transcript_csv": "/path/to/test_transcript.csv"
}

# Create and validate
result = generate_agent_function_code(sentiment_task)
print(result)



```python
import os
import logging

def sentiment_analysis_agent(state: PipelineState) -> PipelineState:
    transc_csv = state.get('transc_csv')
    if not transc_csv or not os.path.isfile(transc_csv):
        logging.error(f"Invalid transcription CSV path for sentiment analysis: {transc_csv}")
        return {"sentiment_analysis_output": "Invalid: No CSV file"}

    task_prompt = f"""You are given a CSV file at path: {transc_csv}. Perform the following steps using a hybrid approach:

Step 1: Use [python_repl] to:
- Import pandas as pd.
- Load the CSV into a DataFrame named df.
- Detect the column that contains transcription text:
    * Look for the first column whose header contains the word "transcript" (case‑insensitive).
    * If none is found, assume the first column after any index column.
- Keep df in memory for the next step.

Step 2: For each row in df, make a direct LLM call (do NOT use python_repl) with the prompt:
\"\"\"Classify the sentiment of the following text as Posit

In [ ]:
# def extract_state_keys_from_spec(task_spec: Dict[str, Any]) -> list:
#     """
#     Use LLM to intelligently extract state keys from input specification.
#     """

#     input_spec = task_spec.get("input_spec", {})

#     extraction_prompt = f"""
# Given this input specification for a task:
# {json.dumps(input_spec, indent=2)}

# And this task description:
# {task_spec.get("description", "")}

# What are the specific state dictionary keys that this function would need to extract from the state?

# For example:
# - If it mentions "csv_path", return ["csv_path"]
# - If it mentions "audio directory", return ["audio_dir"]
# - If it mentions "ground truth file", return ["ground_truth_csv"]

# Return ONLY a Python list of strings with the exact key names, nothing else.
# Example format: ["csv_path", "audio_dir"]
# """

#     response = call_llm("", extraction_prompt, model="openai/gpt-oss-120b", temperature=0)

#     # Parse the response to extract the list
#     try:
#         # Try to evaluate the response as a Python list
#         import ast
#         state_keys = ast.literal_eval(response.strip())
#         return state_keys if isinstance(state_keys, list) else []
#     except:
#         return []

def validate_and_fix_function(function_code: str, task_spec: Dict[str, Any], state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Test the generated function with test data and fix if needed.
    """

    function_name = task_spec.get("name")

    # First, try to execute the function definition
    test_prompt = f"""
First, execute this function definition:

```python
{function_code}
```

Now test it with this test state:
```python
test_state = {json.dumps(state, indent=2)}

# Test the function
try:
    result = {function_name}(test_state)
    print("Function executed successfully!")
    print("Result:", result)
    print("Result type:", type(result))
except Exception as e:
    print("Function execution failed!")
    print("Error:", str(e))
    import traceback
    traceback.print_exc()
```

If there are any errors, identify what went wrong and suggest fixes.
"""

    print(f"Testing function {function_name} with validation data...")
    test_result = agent.invoke(test_prompt)

    # Check if there were errors
    if "❌" in test_result or "Error:" in test_result:
        print("🔧 Function needs fixing, generating corrected version...")

        # Use LLM to fix the function
        fix_prompt = f"""
The function had errors during testing. Here's the original function:

```python
{function_code}
```

Test result and errors:
{test_result}

Test state used:
{json.dumps(state, indent=2)}

Please generate a corrected version of the function that fixes these issues.
Make sure to:
1. Handle the specific errors that occurred
2. Ensure the function works with the provided test state
3. Follow the same pattern as the original
4. Include proper error handling

Generate ONLY the corrected complete function code.
"""

        corrected_code = call_llm("", fix_prompt, model="openai/gpt-oss-20b", temperature=0.1)

        # Test the corrected function
        corrected_test_prompt = f"""
Execute the corrected function:

```python
{corrected_code}
```

Test it again:
```python
test_state = {json.dumps(state, indent=2)}
try:
    result = {function_name}(test_state)
    print("✅ Corrected function works!")
    print("Result:", result)
except Exception as e:
    print("❌ Still has errors:")
    print("Error:", str(e))
```
"""

        final_test_result = agent.invoke(corrected_test_prompt)

        return {
            "function_code": corrected_code,
            "original_code": function_code,
            "test_result": final_test_result,
            "needed_correction": True,
            "success": "✅" in final_test_result
        }

    else:
        return {
            "function_code": function_code,
            "test_result": test_result,
            "needed_correction": False,
            "success": True
        }

def create_and_validate_agent_function(task_spec: Dict[str, Any], validation_state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Create an agent function and validate it with test data.
    """

    function_name = task_spec.get("name")
    print(f"🔨 Creating and validating agent function: {function_name}")

    try:
        # Step 1: Generate the function code using LLM
        print("1️⃣ Generating function code...")
        function_code = generate_agent_function_code(task_spec)

        print("Generated function preview:")
        print("="*50)
        print(function_code[:500] + "..." if len(function_code) > 500 else function_code)
        print("="*50)

        # Step 2: Validate and fix the function
        print("2️⃣ Validating function with test data...")
        validation_result = validate_and_fix_function(function_code, task_spec, validation_state)

        if validation_result["success"]:
            print(f"✅ Function {function_name} created and validated successfully!")
        else:
            print(f"❌ Function {function_name} validation failed")

        return {
            "function_name": function_name,
            "task_spec": task_spec,
            **validation_result
        }

    except Exception as e:
        error_msg = f"Failed to create function {function_name}: {str(e)}"
        print(f"❌ {error_msg}")

        return {
            "function_name": function_name,
            "error": error_msg,
            "success": False
        }

def dynamic_function_creation_and_validation_node(state):
    """
    LangGraph node that creates and validates agent functions dynamically.
    """
    print("---Dynamic Agent Function Creation & Validation---")

    tool_plan = state.get("tool_plan", {})
    created_functions = {}

    # Prepare validation state with test data
    validation_state = {
        "test_audio_dir": state.get("test_audio_dir", ""),
        "test_transcript_csv": state.get("test_transcript_csv", ""),
        "audio_dir": state.get("audio_dir", ""),
        "ground_truth_csv": state.get("ground_truth_csv", ""),
        "lang_code": state.get("lang_code", "en"),
        "csv_path": state.get("test_transcript_csv", "")  # For functions expecting csv_path
    }

    print(f"Using validation state: {list(validation_state.keys())}")

    # Process each task in the plan
    for plan_item in tool_plan.get("PLAN", []):
        task_name = plan_item.get("task")
        create_new_tasks = plan_item.get("create_new", [])

        print(f"\\n📋 Processing task: {task_name}")

        for new_task in create_new_tasks:
            function_name = new_task.get("name")

            # Create and validate the function
            creation_result = create_and_validate_agent_function(new_task, validation_state)
            created_functions[function_name] = creation_result

    return {
        "created_agent_functions": created_functions,
        "validation_state_used": validation_state
    }